This file uses drake to do trajectory optimization (via direct collocation) on the cartpole pendulum. 

In [ ]:
from __future__ import print_function

import pydrake
import math
import numpy as np
import matplotlib.pyplot as plt

from pydrake.all import (DirectCollocation, FloatingBaseType,
                         PiecewisePolynomial, RigidBodyTree, RigidBodyPlant,
                         SolutionResult)


from pydrake.all import (BasicVector, DiagramBuilder, FloatingBaseType,
                         LinearQuadraticRegulator, RigidBodyPlant,
                         RigidBodyTree, Simulator, SignalLogger, LeafSystem, PortDataType, VectorSystem)

from underactuated import (FindResource, PlanarRigidBodyVisualizer)

from IPython.display import HTML

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.constraints import max_norm


%matplotlib inline

tree = RigidBodyTree(FindResource("cartpole/cartpole.urdf"),
                     FloatingBaseType.kFixed)
plant = RigidBodyPlant(tree)
context = plant.CreateDefaultContext()

dircol = DirectCollocation(plant, context, num_time_samples=21,
                           minimum_timestep=0.1, maximum_timestep=0.4)

dircol.AddEqualTimeIntervalsConstraints()

initial_state = (0., 0., 0., 0.)
dircol.AddBoundingBoxConstraint(initial_state, initial_state,
                                dircol.initial_state())
# More elegant version is blocked on drake #8315:
# dircol.AddLinearConstraint(dircol.initial_state() == initial_state)

final_state = (0., math.pi, 0., 0.)
dircol.AddBoundingBoxConstraint(final_state, final_state,
                                dircol.final_state())
# dircol.AddLinearConstraint(dircol.final_state() == final_state)

R = 10  # Cost on input "effort".
u = dircol.input()
dircol.AddRunningCost(R*u[0]**2)

# Add a final cost equal to the total duration.
dircol.AddFinalCost(dircol.time())

initial_x_trajectory = \
    PiecewisePolynomial.FirstOrderHold([0., 4.],
                                       np.column_stack((initial_state,
                                                        final_state)))

dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)

result = dircol.Solve()
assert(result == SolutionResult.kSolutionFound)


x_trajectory = dircol.ReconstructStateTrajectory()
u_trajectory = dircol.ReconstructInputTrajectory()
times = np.linspace(u_trajectory.start_time(), u_trajectory.end_time(), 100)

u_values = u_trajectory.value(0)
for t in times[1:]:
    u_values = np.concatenate((u_values, u_trajectory.value(t)), axis=0)

x_values = x_trajectory.value(0)
for t in times[1:]:
    x_values = np.concatenate((x_values, x_trajectory.value(t)), axis=1)


# sg this was how it was done before, still works fine for u but not for x, changed for consitency
#u_lookup = np.vectorize(u_trajectory.value)
#u_values = u_lookup(times)

plt.figure()
plt.plot(times, u_values)
plt.title("u_trajectory")
plt.xlabel('time (seconds)')
plt.ylabel('force (Newtons)')


prbv = PlanarRigidBodyVisualizer(tree, xlim=[-2.5, 2.5], ylim=[-1, 2.5])
ani = prbv.animate(x_trajectory, resample=30, repeat=True)
plt.close(prbv.fig)
HTML(ani.to_html5_video())


#plt.show()

In [ ]:
# create model
# This is the actual 3 layer MLP, find it as "policy" using tensorboard 
with tf.variable_scope('pi/pol/'):

    model = Sequential()
    model.add(Dense(12, input_dim=4, kernel_initializer='normal', activation='relu', name = 'fc1'))
    model.add(Dense(12,   input_dim=12, kernel_initializer='normal', activation='relu', name = 'fc2'))
    #model.add(Dense(32,   input_dim=32, kernel_initializer='normal', activation='relu', name = 'fc3'))
    model.add(Dense(1, kernel_initializer='normal', kernel_constraint=max_norm(10.), name = 'final' ))

    
# Compile model
# This adds some other stuff to your model that we don't want to consider the policy
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# you can either go ahead and fit the model or load weights, this cell will go ahead and run the fit routine

history = model.fit(x_values.T, u_values, epochs=20, verbose=0)

plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
class SwingupController(LeafSystem):
    def __init__(self, rbt,
                 control_period=0.005,
                 print_period=0.5):
        LeafSystem.__init__(self)
        self.set_name("Swing up Controller")


       
        #self.B_inv = np.linalg.inv(self.B)
        # Copy lots of stuff
        self.rbt = rbt
        self.nq = rbt.get_num_positions()
        #self.plant = plant
        self.nu = rbt.get_input_port(0).size()
        #self.print_period = print_period
        self.last_print_time = -print_period
        self.shut_up = False

        self.robot_state_input_port = \
            self._DeclareInputPort(PortDataType.kVectorValued,
                                   rbt.get_num_positions() +
                                   rbt.get_num_velocities())
            

        self._DeclareContinuousState(self.nu)
        #self._DeclarePeriodicContinuousUpdate(period_sec=control_period)
        self._DeclareVectorOutputPort(
            BasicVector(self.nu),
            self._DoCalcVectorOutput)
        
    def _DoCalcVectorOutput(self, context, y_data):
        control_output = context.get_continuous_state_vector().get_value()
        y = y_data.get_mutable_value()
        # Get the ith finger control output
        # y[:] = control_output[:]
        y[:] = 0

In [ ]:
class Controller(VectorSystem):
    """ Defines a feedback controller for the double pendulum.

    The controller applies torques at the joints in order to
    1) cancel out the dynamics of the double pendulum,
    2) make the first joint swing with the dynamics of a single pendulum, and
    3) drive the second joint towards zero.

    The magnitude of gravity for the imposed single pendulum dynamics is taken
    as a constructor argument.  So you can do fun things like pretending that
    gravity is zero, or even inverting gravity!

    """

    def __init__(self, rigid_body_tree):
        # 4 inputs (double pend state), 2 torque outputs.
        VectorSystem.__init__(self, 4, 1)
        self.tree = rigid_body_tree

        
        
    def _DoCalcVectorOutput(self, context, x, u, y):
        print(context)
        print(u)
        print(x)
        print(y)
        u = 0
        return 0
        
        
    
        
    #def _DoCalcVectorOutput(self, context, y_data,w,t):
        #print(context)
        #print(y_data)
        #print(w)
        #print(t)
     #   control_output = context.get_continuous_state_vector().get_value()
        
        # Get the ith finger control output
        # y[:] = control_output[:]
     #   y_data[:] = 0
     #   return y_data


In [ ]:
#%tb
# Define an upright state
def UprightState():
    state = (0,math.pi,0,0)
    return state

def UprightPos():
    state = (math.pi/2,0)
    return state


tree = RigidBodyTree(FindResource("cartpole/cartpole.urdf"),
                     FloatingBaseType.kFixed)

builder = DiagramBuilder()

robot = builder.AddSystem(RigidBodyPlant(tree))

#controller = builder.AddSystem(Controller(robot))
controller = builder.AddSystem(Controller(robot))

#other_controller = builder.AddSystem(SwingupController(robot))

builder.Connect(robot.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), robot.get_input_port(0))

logger = builder.AddSystem(SignalLogger(robot.get_output_port(0).size()))
logger._DeclarePeriodicPublish(1. / 30., 0.0)
builder.Connect(robot.get_output_port(0), logger.get_input_port(0))

diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False)
context = simulator.get_mutable_context()

state = context.get_mutable_continuous_state_vector()
state.SetFromVector(UprightState() + 0.1*np.random.randn(4,))
#state.SetFromVector(UprightState)
simulator.StepTo(10.)

prbv = PlanarRigidBodyVisualizer(tree, xlim=[-2.5, 2.5], ylim=[-1, 2.5])
ani = prbv.animate(logger, resample=30, repeat=True)
plt.close(prbv.fig)
HTML(ani.to_html5_video())